<a href="https://colab.research.google.com/github/hannahpu/widsdatathon2022/blob/main/notebook/model_run_hannah_stack_regressor_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# repo: https://github.com/hannahpu/widsdatathon2022
! git clone https://github.com/hannahpu/widsdatathon2022.git

Cloning into 'widsdatathon2022'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 354 (delta 197), reused 283 (delta 136), pack-reused 0
Receiving objects: 100% (354/354), 27.06 MiB | 12.50 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [ ]:
# # run this line of code when there is update in git repo
# %cd /content/widsdatathon2022
# ! git pull

In [ ]:
!pip install catboost
!pip install xgboost
!pip install mlxtend
!pip install imblearn
!pip install nb-black

In [ ]:
# lightgbm gpu https://medium.com/@am.sharma/lgbm-on-colab-with-gpu-c1c09e83f2af
# https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html#build-gpu-version
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install — precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 25053, done.
remote: Counting objects: 100% (729/729), done.
remote: Compressing objects: 100% (402/402), done.
remote: Total 25053 (delta 469), reused 520 (delta 320), pack-reused 24324
Receiving objects: 100% (25053/25053), 18.65 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (18418/18418), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/python-package/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), 

In [ ]:
import sys

import pandas as pd
import catboost as cb
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor 
from mlxtend.regressor import StackingRegressor
import imblearn
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import lightgbm as lgb
import numpy as np
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

wids_path = "/content/widsdatathon2022/"
sys.path.append(wids_path)
import global_vars as gv
from utils import model_utils as mu
from utils import data_utils as du
from utils import data_process_utils as dpu
from utils import visualize as viz

In [ ]:
#### read in data
test_df = pd.read_csv("/content/widsdatathon2022/data/test.csv")
print(f"Test dimension: {test_df.shape}")
train_df = pd.read_csv("/content/widsdatathon2022/data/train.csv")
print(f"Train dimension: {train_df.shape}")
sample_solution_df = pd.read_csv("/content/widsdatathon2022/data/sample_solution.csv")
print(f"Sample solution dimension: {sample_solution_df.shape}")
train_df.columns = train_df.columns.str.lower()
test_df.columns = test_df.columns.str.lower()

Test dimension: (9705, 63)
Train dimension: (75757, 64)
Sample solution dimension: (9705, 2)


In [ ]:
# Memory optimization

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16
def reduce_mem_usage(data, use_float16=False) -> pd.DataFrame:
    start_mem = data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in data.columns:
        if is_datetime(data[col]) or is_categorical_dtype(data[col]):
            continue
        col_type = data[col].dtype

        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)
        else:
            data[col] = data[col].astype('category')

    end_mem = data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.2f}%'.format(
        100 * (start_mem - end_mem) / start_mem))

    return data

In [ ]:
# Memory optimization
train_df = reduce_mem_usage(train_df, use_float16=True)
test_df = reduce_mem_usage(test_df, use_float16=True)

Memory usage of dataframe is 36.99 MB
Memory usage after optimization is: 7.16 MB
Decreased by 80.66%
Memory usage of dataframe is 4.66 MB
Memory usage after optimization is: 0.87 MB
Decreased by 81.29%


In [ ]:
train_w_parsed_facility_type_df = dpu.parse_facility_type(
        input_df=train_df.copy(),
        facility_type_colname="facility_type")
test_w_parsed_facility_type_df = dpu.parse_facility_type(
        input_df=test_df.copy(),
        facility_type_colname="facility_type")

In [ ]:
groupby_list = ["state_factor", "building_class", "facility_type"]
col = "energy_star_rating"
train_backfill_energy_star_rating_df = dpu.backfill_energy_star_rating(
                input_df=train_df,
                mapping_df=train_df,
                groupby_list=groupby_list,
                energy_star_rating_colname=col,
                agg_approach_func=np.nanmedian,
            )
test_backfill_energy_star_rating_df = dpu.backfill_energy_star_rating(
                input_df=test_df,
                mapping_df=train_df,
                groupby_list=groupby_list,
                energy_star_rating_colname=col,
                agg_approach_func=np.nanmedian,
            )

In [ ]:
groupby_list = ["state_factor", "building_class", "facility_type_parsed"]
col = "energy_star_rating"
train_backfill_energy_star_rating_df_v1 = dpu.backfill_energy_star_rating(
                input_df=train_w_parsed_facility_type_df,
                mapping_df=train_w_parsed_facility_type_df,
                groupby_list=groupby_list,
                energy_star_rating_colname=col,
                agg_approach_func=np.nanmedian,
            ).rename(columns = {"backfilled_energy_star_rating": "backfilled_energy_star_rating_v1"})
test_backfill_energy_star_rating_df_v1 = dpu.backfill_energy_star_rating(
                input_df=test_w_parsed_facility_type_df,
                mapping_df=train_w_parsed_facility_type_df,
                groupby_list=groupby_list,
                energy_star_rating_colname=col,
                agg_approach_func=np.nanmedian,
            ).rename(columns = {"backfilled_energy_star_rating": "backfilled_energy_star_rating_v1"})

In [ ]:
# add back to train
train_filled_df = train_df.merge(
    train_backfill_energy_star_rating_df[["id", "backfilled_energy_star_rating"]],
    on = "id",
    how = "left"
).merge(
    train_w_parsed_facility_type_df[["id", "facility_type_parsed"]],
    on = "id",
    how = "left"
).merge(
    train_backfill_energy_star_rating_df_v1[["id", "backfilled_energy_star_rating_v1"]],
    on = "id",
    how = "left"
)
print(train_df.shape)
print(train_filled_df.shape)
display(train_filled_df[["id", "energy_star_rating","backfilled_energy_star_rating","backfilled_energy_star_rating_v1",  "facility_type","facility_type_parsed"]].notnull().sum())
assert train_filled_df.shape[0] == train_df.shape[0]

(75757, 64)
(75757, 67)


id                                  75757
energy_star_rating                  49048
backfilled_energy_star_rating       73491
backfilled_energy_star_rating_v1    75239
facility_type                       75757
facility_type_parsed                75757
dtype: int64

In [ ]:
train_filled_df = train_filled_df.assign(resample_priority = train_filled_df["energy_star_rating"].notnull())
weighted_dict = {True: 1, False: 0.5}
train_filled_df["resample_weights"] = train_filled_df["resample_priority"].replace(weighted_dict)
# train_filled_df = train_filled_df.assign(resample_weights = train_filled_df["resample_priority"].astype(int))

In [ ]:
# add back to test
test_filled_df = test_df.merge(
    test_backfill_energy_star_rating_df[["id", "backfilled_energy_star_rating"]],
    on = "id",
    how = "left"
).merge(
    test_w_parsed_facility_type_df[["id", "facility_type_parsed"]],
    on = "id",
    how = "left"
).merge(
    test_backfill_energy_star_rating_df_v1[["id", "backfilled_energy_star_rating_v1"]],
    on = "id",
    how = "left"
)
print(test_df.shape)
print(test_filled_df.shape)
display(test_filled_df[["id", "energy_star_rating","backfilled_energy_star_rating", "backfilled_energy_star_rating_v1","facility_type","facility_type_parsed"]].notnull().sum())
assert test_filled_df.shape[0] == test_df.shape[0]

(9705, 63)
(9705, 66)


id                                  9705
energy_star_rating                  7451
backfilled_energy_star_rating       9153
backfilled_energy_star_rating_v1    9546
facility_type                       9705
facility_type_parsed                9705
dtype: int64

## Model config

In [ ]:
cat = CatBoostRegressor(
                    iterations=3000,
                    grow_policy='Lossguide',
                    verbose=False,
                    task_type='GPU',
                    l2_leaf_reg=1,
                    learning_rate=0.1,
                    depth= 10,
                )
xgb_model = XGBRegressor(
                    grow_policy='lossguide',
                    tree_method='gpu_hist',
                    n_estimators=3000,
                    eta=0.1, 
                    max_depth= 10,
                    reg_lambda=1 #
                )
lgbm_model = LGBMRegressor(
    
                                max_depth = 11,
                                num_leaves = 15,
                                learning_rate = 0.05,
                                subsample = 0.9,
                                colsample_bytree = 0.7,
                    
                    device="gpu",
                  
                    reg_lambda=10 
                )
knn = KNeighborsRegressor(n_neighbors=20)
svr = SVR(C=1.0, epsilon=0.2)
mlp = MLPRegressor(hidden_layer_sizes=(20,20,20),random_state=1, max_iter=500)
lightgbm = LGBMRegressor(objective='regression', learning_rate=0.05, num_leaves=1024,
device="gpu",
    feature_fraction=0.8, bagging_fraction=0.9, bagging_freq=5) 

ridge = Ridge(alpha=10)
lasso = Lasso(alpha=0.3)

In [ ]:
seed = 0
depth = 12

model_dict = {
    "catboost": cb.CatBoostRegressor(
                loss_function="RMSE",
                depth=depth,
                random_seed=seed,
                verbose=False,
                nan_mode="Min",
            ),
    # "stackingregressor": StackingRegressor(regressors=(lgbm_model, cat, xgb_model, knn, ridge, svr),  #from mlx library
    #     meta_regressor=xgb_model, use_features_in_secondary=True, verbose = 2),
    "stackingregressor": StackingRegressor(regressors=(cat, xgb_model, knn, ridge, svr),  #from mlx library
        meta_regressor=xgb_model, use_features_in_secondary=True, verbose = 2),
    "xgboost": xgb.XGBRegressor(eval_metric= 'rmse', seed = seed, max_depth=11, n_estimators=1000, 
                           booster='gbtree', n_jobs=-1,num_leaves = 15,subsample = 0.9,
                                colsample_bytree = 0.7,
                           random_state=0, learning_rate=0.05),

    "lightgbm": lgb.LGBMRegressor (
                                n_estimators = 30000,
                                max_depth = 11,
                                num_leaves = 15,
                                learning_rate = 0.05,
                                subsample = 0.9,
                                colsample_bytree = 0.7,
                                random_state = 42 )

}

model_type_dict = {
    "catboost": "catboost",
    "xgboost": "sklearn",
    "lightgbm": "lightgbm",
    "stackingregressor": "sklearn"
}

feature_dict = {
   "log_temp_pca_onehot_impute_parse_stacking": {
        "cols_to_reduce_dict": {"temp": viz.temp_col_list},
        "reduce_number_dict": {"temp": 9},
        "log10_transform_cols": ["floor_area"],
        "if_one_hot": True,
        "if_scale": False,
        "model": "stackingregressor",
        "replace_original_feature_col_dict": {"energy_star_rating": "backfilled_energy_star_rating", "facility_type": "facility_type_parsed"},
        "drop_data": {},
        "resample_param_dict": {}
    },
    "log_temp_pca_onehot_impute_parse_upsample_stacking": {
        "cols_to_reduce_dict": {"temp": viz.temp_col_list},
        "reduce_number_dict": {"temp": 9},
        "log10_transform_cols": ["floor_area"],
        "if_one_hot": True,
        "if_scale": False,
        "model": "stackingregressor",
        "replace_original_feature_col_dict": {"energy_star_rating": "backfilled_energy_star_rating", "facility_type": "facility_type_parsed"},
        "drop_data": {},
        "resample_param_dict": {"up_or_downsample": "upsample", "resample_by_col": "state_factor", "resample_type": "random"}
    },
    "log_temp_pca_onehot_impute_parse_upsample_lightgbm": {
        "cols_to_reduce_dict": {"temp": viz.temp_col_list},
        "reduce_number_dict": {"temp": 9},
        "log10_transform_cols": ["floor_area"],
        "if_one_hot": True,
        "if_scale": False,
        "model": "lightgbm",
        "replace_original_feature_col_dict": {"energy_star_rating": "backfilled_energy_star_rating", "facility_type": "facility_type_parsed"},
        "drop_data": {},
        "resample_param_dict": {"up_or_downsample": "upsample", "resample_by_col": "state_factor", "resample_type": "random"}
    }
}

In [ ]:
model_configs_to_run =["log_temp_pca_onehot_impute_parse_upsample_stacking", "log_temp_pca_onehot_impute_parse_stacking"]

In [ ]:
for config_name in model_configs_to_run:
# config_name = "log_temp_pca_onehot_impute_catboost"
    # get model config
    cols_to_reduce_dict = feature_dict[config_name]["cols_to_reduce_dict"]
    reduce_number_dict = feature_dict[config_name]["reduce_number_dict"]
    log10_transform_cols = feature_dict[config_name]["log10_transform_cols"]
    if_one_hot = feature_dict[config_name]["if_one_hot"]
    if_scale = feature_dict[config_name]["if_scale"]
    replace_original_feature_col_dict = feature_dict[config_name]["replace_original_feature_col_dict"]
    resample_param_dict = feature_dict[config_name]["resample_param_dict"]
    drop_data_dict = feature_dict[config_name]["drop_data"]
    model_name = feature_dict[config_name]["model"]
    model = model_dict[model_name]
    # check if need to drop data
    if len(drop_data_dict) > 0:
        for one_col, drop_level_list in drop_data_dict.items():
            train_filled_df = train_filled_df.query(f"{one_col} not in {drop_level_list}")
    # process data
    train_filter_df, test_filter_df, pca_cols = du.process_data_v1(
        train_filled_df.drop_duplicates(),
        test_filled_df.drop_duplicates(),
        reduce_col_dict = cols_to_reduce_dict,
        cols_to_log_transform = log10_transform_cols,
        reduce_number_dict = reduce_number_dict,
    )
    # Set feature columns after data transformations
    all_cols_to_reduce = []
    for _, one_set_col_to_reduce in cols_to_reduce_dict.items():
        all_cols_to_reduce = all_cols_to_reduce + one_set_col_to_reduce

    all_cols_to_drop = []
    all_cols_to_replace_from_drop = []
    for col_to_drop, col_to_replace in replace_original_feature_col_dict.items():
        all_cols_to_drop.append(col_to_drop)
        all_cols_to_replace_from_drop.append(col_to_replace)

    features_columns = (
        list(set(gv.all_feature_columns) - set(all_cols_to_reduce) - set(log10_transform_cols) - set(all_cols_to_drop))
        + pca_cols
        + all_cols_to_replace_from_drop
        + [f"log10_{col}" for col in log10_transform_cols]
    )
    print(config_name, model_name, features_columns, if_one_hot)

    # run model
    ## Run LOY model
    model_rmse = mu.run_leave_year_out(
        model_df=train_filter_df,
        ml_model=model,
        features_columns=features_columns,
        if_scale_data=if_scale,
        if_one_hot=if_one_hot,
        model_type=model_type_dict[model_name],
        resample_param_dict = resample_param_dict
    )
    print(f"Average RMSE:\n{model_rmse.mean()}")
    display(model_rmse)

    ## predict on test data    
    train_filter_x_df, train_filter_y_df = mu.split_model_feature_response(
        train_filter_df, features_columns
    )
    test_filter_x_df = mu.split_model_feature_response(
        test_filter_df, features_columns, if_with_response=False
    )
    if len(resample_param_dict) > 0:
        train_for_resample_df = train_filter_x_df
        train_for_resample_df["site_eui"] = train_filter_y_df
        resample_by_col = resample_param_dict["resample_by_col"]
        resample_type = resample_param_dict["resample_type"]
        if resample_param_dict["up_or_downsample"] == "upsample":
            train_after_resampled_df = mu.upsampling_by_column(
                train_for_resample_df, resample_by_col, resample_type=resample_type
            )
        elif resample_param_dict["up_or_downsample"] == "downsample":
            train_after_resampled_df = mu.downsampling_by_column(
                train_for_resample_df, resample_by_col, resample_type=resample_type
            )
        elif resample_param_dict["up_or_downsample"] == "custom_upsample":
            print("getting custom upsample")
            train_after_resampled_df = mu.custom_weighted_upsample(train_for_resample_df, train_filter_df, resample_by_col)
        train_filter_x_df, train_filter_y_df = mu.split_model_feature_response(
                    train_after_resampled_df,
                    features_columns,
                    if_with_response=True,
                    response_col="site_eui",
                )

    
    processed_train_x_df, processed_test_x_df = mu.process_train_test_data(
        train_filter_x_df, test_filter_x_df, if_scale, if_one_hot, full_data_df = pd.concat([train_filter_x_df,test_filter_x_df])
    )
    ##
    # Missing data imputation
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    imputer.fit(processed_train_x_df)
    processed_train_x_df = pd.DataFrame(imputer.transform(processed_train_x_df))
    processed_test_x_df = pd.DataFrame(imputer.transform(processed_test_x_df))
    ##
    # run_model_dict = {"xgboost": mu.run_sklearn_model, "catboost": mu.run_catboost_model}
    run_model_dict = {
        "sklearn": mu.run_sklearn_model,
        "catboost": mu.run_catboost_model,
        "lightgbm": mu.run_lgb_model,
        "stackingregressor": mu.run_sklearn_model,
    }
    train_predict, test_predict, fitted_model = run_model_dict[model_name](
            model, processed_train_x_df, train_filter_y_df, processed_test_x_df
        )
    training_rmse = mu.calculate_rmse(train_filter_y_df, train_predict)
    print(f"Whole data train RMSE: {training_rmse}")

    ## output save result
    model_rmse.loc[model_rmse.shape[0], :] = [0, training_rmse, np.nan]
    model_rmse["method"] = model_rmse["left_out_year"].apply(
        lambda x: "loyo" if x > 0 else "whole train"
    )
    display(model_rmse)
    model_rmse.to_csv(
        f"{wids_path}validation_result/{config_name}.csv", index=False
    )
    test_prediction_result = test_df[["id"]]
    test_prediction_result["site_eui"] = test_predict
    test_prediction_result.to_csv(f"{wids_path}prediction_result/{config_name}.csv", index=False)

Fitting PCA with 9 components
log_temp_pca_onehot_impute_parse_upsample_stacking stackingregressor ['direction_peak_wind_speed', 'days_above_100f', 'days_above_90f', 'max_wind_speed', 'days_with_fog', 'snowdepth_inches', 'snowfall_inches', 'building_class', 'precipitation_inches', 'year_built', 'cooling_degree_days', 'days_below_10f', 'days_above_80f', 'state_factor', 'direction_max_wind_speed', 'heating_degree_days', 'days_below_30f', 'days_below_20f', 'days_above_110f', 'elevation', 'days_below_0f', 'temp_pca1', 'temp_pca2', 'temp_pca3', 'temp_pca4', 'temp_pca5', 'temp_pca6', 'temp_pca7', 'temp_pca8', 'temp_pca9', 'backfilled_energy_star_rating', 'facility_type_parsed', 'log10_floor_area'] True
Running sklearn
Modeling 1...
Fitting 5 regressors...
Fitting regressor1: catboostregressor (1/5)
Fitting regressor2: xgbregressor (2/5)
XGBRegressor(eta=0.1, grow_policy='lossguide', max_depth=10, n_estimators=3000,
             tree_method='gpu_hist')
[03:51:48] WARNING: /workspace/src/objec